[Reference](https://python.plainenglish.io/how-to-build-a-data-pipeline-for-api-integration-using-python-and-postgresql-d61a637e7a77)

# Step 1: Setting Up Your Environment

In [1]:
pip install requests psycopg2 pandas

# Setting Up PostgreSQL Database

```
CREATE DATABASE api_data;
```

# Step 2: Fetching Data from an API

In [2]:
import requests

# Define the API endpoint
api_url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 35.6895,
    "longitude": 139.6917,
    "hourly": "temperature_2m"
}

# Make a GET request to fetch data
response = requests.get(api_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print("Data fetched successfully!")
else:
    print("Failed to fetch data:", response.status_code)

Data fetched successfully!


# Step 3: Transforming Data for Storage

In [3]:
import pandas as pd

# Extract the relevant data from the JSON response
temperature_data = data['hourly']['temperature_2m']
timestamps = data['hourly']['time']

# Create a DataFrame
df = pd.DataFrame({'timestamp': timestamps, 'temperature': temperature_data})

# Convert timestamp to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])
print(df.head())

            timestamp  temperature
0 2024-11-30 00:00:00          9.0
1 2024-11-30 01:00:00         11.4
2 2024-11-30 02:00:00         12.9
3 2024-11-30 03:00:00         13.6
4 2024-11-30 04:00:00         14.2


# Step 4: Creating a Table in PostgreSQL

```
CREATE TABLE weather_data (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP,
    temperature FLOAT
);
```

# Step 5: Loading Data into PostgreSQL

In [5]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="api_data", user="your_user", password="your_password", host="localhost"
)
cur = conn.cursor()

# Insert DataFrame into PostgreSQL
for _, row in df.iterrows():
    cur.execute(
        "INSERT INTO weather_data (timestamp, temperature) VALUES (%s, %s)",
        (row['timestamp'], row['temperature'])
    )

# Commit changes and close connection
conn.commit()
cur.close()
conn.close()
print("Data inserted into PostgreSQL successfully!")

# Step 6: Automating the Data Pipeline

## 1. Open the crontab editor:
```
crontab -e
```

## 2. Schedule your script to run every day at midnight:
```
0 0 * * * /usr/bin/python3 /path/to/data_pipeline.py
```

# Step 7: Querying Data for Analysis

In [6]:
conn = psycopg2.connect(
    dbname="api_data", user="your_user", password="your_password", host="localhost"
)
cur = conn.cursor()

cur.execute("SELECT * FROM weather_data WHERE timestamp > NOW() - INTERVAL '1 day'")
rows = cur.fetchall()

for row in rows:
    print(row)

cur.close()
conn.close()